<table align="center" width=100%>
    <tr>
        <td width="10%">
            <img src="in_class.png">
        </td>
        <td>
            <div align="center">
                <font color="#21618C" size=8px>
                    <b> Inclass-Lab  <br>(Day 4)
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

## Table of Content

1. **[Chi-Square Test](#chisq)**
2. **[One-way ANOVA](#1way)**
3. **[Equivalent Non-parametric Test](#non-para)**
4. **[Two-way ANOVA](#2way)**

**Import the required libraries**

In [1]:
# import 'pandas' 
import pandas as pd 

# import 'numpy' 
import numpy as np

# import subpackage of matplotlib
import matplotlib.pyplot as plt

# import 'seaborn'
import seaborn as sns

# to suppress warnings 
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# import 'stats' package from scipy library
from scipy import stats

# import the functions to perform Chi-square tests
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy.stats import chisquare

# function to perform post-hoc test
import statsmodels.stats.multicomp as mc

# import function to perform post-hoc
# install scikit_posthocs using "!pip install scikit_posthocs" 
import scikit_posthocs

In [2]:
# set the plot size using 'rcParams'
# once the plot size is set using 'rcParams', it sets the size of all the forthcoming plots in the file
# pass width and height in inches to 'figure.figsize' 
plt.rcParams['figure.figsize'] = [15,8]

### Let's begin with some hands-on practice exercises

<a id = "chisq"> </a>
## 1. Chi-Square Test

Use the data available in the CSV file `Employee_Attrition.csv` for questions 1 to 8.

In [3]:
# read the sample data 
df_emp = pd.read_csv('Employee_Attrition.csv')

# display the first two observations
df_emp.head(2)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>1. A company in Los Angeles has three functional departments - Research and Development, Sales, and Human Resources. The company claims that the percentage of employees in these 3 departments is 55%, 35% and 10% respectively. Check the company's claim using p-value criteria. Consider a 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: There is no significant difference between the observed and expected values. <br>
H<sub>1</sub>: There is a significant difference between the observed and expected values.

In [4]:
# observed values
df_emp['Department'].value_counts()

Research & Development    961
Sales                     446
Human Resources            63
Name: Department, dtype: int64

In [5]:
# given observed values
observed_value = [961, 446, 63]

# expected percentage 
exp_count = [0.55, 0.35, 0.10]

# calculate the expected values for each category
expected_value = np.array(exp_count) * len(df_emp)

# round-off the values to integers
expected_value = round(pd.Series(expected_value))

# use the 'chisquare()' to perform the goodness of fit test
# the function returns the test statistic value and corresponding p-value
# pass the observed values to the parameter, 'f_obs'
# pass the expected values to the parameter, 'f_exp'
stat, p_value = chisquare(f_obs = observed_value, f_exp = expected_value)

print('p-value:', p_value)

p-value: 2.642446152388858e-19


The above output shows that the p-value is less than 0.05. Thus, we reject the null hypothesis and conclude that the company's claim is not correct. i.e. the distribution of employees in a company is not as per the company's claim.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>2. The employees in an IT firm undergo an online assessment survey. The survey reveals that 20% of employees are least satisfied, 18% are fairly satisfied, 30% are moderately satisfied and 32% are highly satisfied with their job. Use a critical value method to test the survey result with 90% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: There is no significant difference between the observed and expected values. <br>
H<sub>1</sub>: There is a significant difference between the observed and expected values.

In [6]:
# observed values
df_emp['JobSatisfaction'].value_counts()

4    459
3    442
1    289
2    280
Name: JobSatisfaction, dtype: int64

As the job satisfaction level is categorical; change the data type of the variable to `object`.

In [7]:
df_emp['JobSatisfaction'] = df_emp['JobSatisfaction'].astype('object')

For ⍺ = 0.1 and degrees of freedom (= 4-1) = 3, calculate the critical value.

In [8]:
# calculate the χ2-value for 90% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'
# pass the value of 'alpha' to the parameter 'q', here alpha = 0.1
# pass the degrees of freedom to the parameter 'df' 
# use 'round()' to round-off the value to 4 digits
chi2_val = np.abs(round(stats.chi2.isf(q = 0.1, df = 3), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 6.2514


i.e. if the chi-square value is greater than 6.2514 then we reject the null hypothesis.

In [9]:
# observed values
observed_value = [289, 280, 442, 459]

# expected percentage 
exp_count = [0.2, 0.18, 0.3, 0.32]

# calculate the expected values for each category
expected_value = np.array(exp_count) * len(df_emp)

# round-off the values to integers
expected_value = round(pd.Series(expected_value))

# use the 'chisquare()' to perform the goodness of fit test
# the function returns the test statistic value and corresponding p-value
# pass the observed values to the parameter, 'f_obs'
# pass the expected values to the parameter, 'f_exp'
stat, p_value = chisquare(f_obs = observed_value, f_exp = expected_value)

print('Test statistic:', stat)

Test statistic: 1.1938049995858107


The above output shows that the test statistic is less than 6.2514. Thus, we fail to reject (i.e. accept) the null hypothesis and conclude that there is no difference in the sample percentage and the percentage revealed by the survey.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>3. The company claims that in the organization, 68% of employees are there with rare travelling, 20% with frequent travelling and 12% of employees does not need to travel for business. Check if the given data fits with the company's claimed distribution. Use the p-value technique with 99% confidence. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: There is no significant difference between the observed and expected values. <br>
H<sub>1</sub>: There is a significant difference between the observed and expected values.

In [10]:
# observed values
df_emp['BusinessTravel'].value_counts()

Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: BusinessTravel, dtype: int64

In [11]:
# given observed values
observed_value = [1043, 277, 150]

# expected percentage 
exp_count = [0.68, 0.20, 0.12]

# calculate the expected values for each category
expected_value = np.array(exp_count) * len(df_emp)

# round-off the values to integers
expected_value = round(pd.Series(expected_value))

# use the 'chisquare()' to perform the goodness of fit test
# the function returns the test statistic value and corresponding p-value
# pass the observed values to the parameter, 'f_obs'
# pass the expected values to the parameter, 'f_exp'
stat, p_value = chisquare(f_obs = observed_value, f_exp = expected_value)

print('p-value:', p_value)

p-value: 0.03556294179573812


The above output shows that the p-value is greater than 0.01. Thus, we fail to reject (i.e. accept) the null hypothesis and conclude that company's claim is correct. i.e. there is no significant difference between the observed and expected values.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>4. Check whether travelling for work depends upon the job role of an employee. Use p-value criteria to test the dependence with 99% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: Business travel and job role are independent<br>
H<sub>1</sub>: Business travel and job role are not independent

In [12]:
# use 'crosstab()' to create a table for each type of business travel and job role 
table = pd.crosstab(df_emp['BusinessTravel'], df_emp['JobRole'])

# observed values  
observed_value = table.values
observed_value

array([[ 15,   4,  28,  12,  13,   6,  28,  39,   5],
       [ 26,  10,  51,  13,  29,  12,  54,  59,  23],
       [ 90,  38, 180,  77, 103,  62, 210, 228,  55]], dtype=int64)

In [13]:
# use the 'chi2_contingency()' to check the independence of variables
# the function returns the test statistic value, corresponding p-value, degrees of freedom of the test and expected values
# pass the observed values to the parameter, 'observed'
# 'correction = False' will not apply the Yates' correction
test_stat, p, dof, expected_value = chi2_contingency(observed = observed_value, correction = False)

print("p-value:", p)

p-value: 0.7448263418408124


The above output shows that the p-value is greater than 0.01, thus we fail to reject (i.e. accept) the null hypothesis and conclude that the business travel is not dependent on the job role.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>5. Is there any relationship between the attrition of an employee and his/her marital status? Use the critical value technique to test the relationship with 95% confidence. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: Attrition and marital status are independent<br>
H<sub>1</sub>: Attrition and marital status are not independent

In [14]:
# use 'crosstab()' to create a table for each attrition level and marital status 
table = pd.crosstab(df_emp['Attrition'], df_emp['MaritalStatus'])

# observed values  
observed_value = table.values
observed_value

array([[294, 589, 350],
       [ 33,  84, 120]], dtype=int64)

For ⍺ = 0.05 and degrees of freedom (= 1x2) = 2, calculate the critical value.

In [15]:
# calculate the χ2-value for 95% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'
# pass the value of 'alpha' to the parameter 'q', here alpha = 0.05
# pass the degrees of freedom to the parameter 'df' 
# use 'round()' to round-off the value to 4 digits
chi2_val = np.abs(round(stats.chi2.isf(q = 0.05, df = 2), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 5.9915


i.e. if the chi-square value is greater than 5.9915 then we reject the null hypothesis.

In [16]:
# use the 'chi2_contingency()' to check the independence of variables
# the function returns the test statistic value, corresponding p-value, degrees of freedom of the test and expected values
# pass the observed values to the parameter, 'observed'
# 'correction = False' will not apply the Yates' correction
test_stat, p, dof, expected_value = chi2_contingency(observed = observed_value, correction = False)

print("Test statistic:", test_stat)

Test statistic: 46.163676540848705


The above output shows that the test statistic is greater than 5.9915, thus we reject the null hypothesis and conclude that attrition and marital status are dependent.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>6. Check whether travelling for the business depends on the gender of an employee. Use the p-value technique to test the claim with 90% confidence. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: Business travel and gender of an employee are independent<br>
H<sub>1</sub>: Business travel and gender of an employee are not independent

In [17]:
# use 'crosstab()' to create a table for each type of business travel and gender 
table = pd.crosstab(df_emp['BusinessTravel'], df_emp['Gender'])

# observed values  
observed_value = table.values
observed_value

array([[ 49, 101],
       [117, 160],
       [422, 621]], dtype=int64)

In [18]:
# use the 'chi2_contingency()' to check the independence of variables
# the function returns the test statistic value, corresponding p-value, degrees of freedom of the test and expected values
# pass the observed values to the parameter, 'observed'
# 'correction = False' will not apply the Yates' correction
test_stat, p, dof, expected_value = chi2_contingency(observed = observed_value, correction = False)

print("p-value:", p)

p-value: 0.13322895625828154


The above output shows that the p-value is greater than 0.1, thus we fail to reject (i.e. accept) the null hypothesis and conclude that the business travel is independent of the gender of an employee.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>7. The economic forum claims that the variance in the percentage salary hike is more the 12% in Manhattan. To test the forum's claim use the data obtained from a telecom company in Manhattan. Use a critical value method to test the claim with 90% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: $\sigma^{2} \leq 12$<br>
H<sub>1</sub>: $\sigma^{2} > 12$

In [19]:
# given data
salary = df_emp['PercentSalaryHike']

# hypothesized variance
sig_2 = 12

# sample size
n = len(salary)

# the population mean is unknown, use the sample mean
samp_mean = np.mean(salary)

# degrees of freedom
print('Degrees of freedom:', n-1)

Degrees of freedom: 1469


For ⍺ = 0.1 and degrees of freedom = 1469, calculate the critical value for the right-tail.

In [20]:
# calculate the χ2-value for 90% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'
# pass the value of 'alpha' to the parameter 'q'
# pass the degrees of freedom to the parameter 'df' 
# use 'round()' to round-off the value to 4 digits
chi2_val = np.abs(round(stats.chi2.isf(q = 0.1, df = 1469), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 1538.8785


i.e. if the chi-square value is greater than 1538.8785 then we reject the null hypothesis.

In [21]:
# calculate the test statistic
chi_test = (np.sum((salary - samp_mean)**2)) / sig_2

# print the test statistic
print('Test Statistic:', chi_test)

Test Statistic: 1639.788888888889


The above output shows that the chi-square test statistic is greater than 1538.8785. Thus we reject the null hypothesis and conclude that the variance in the percentage salary hike is more the 12% in Manhattan. 

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>8. The worker's association claims that the distance travelled by each employee to reach the office has a variance of less than 15 miles. Use a critical value method to test the claim with 95% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: $\sigma^{2} \geq 15$<br>
H<sub>1</sub>: $\sigma^{2} < 15$

In [22]:
# given data
dist = df_emp['DistanceFromHome']

# hypothesized variance
sig_2 = 15

# sample size
n = len(dist)

# the population mean is unknown, use the sample mean
samp_mean = np.mean(dist)

# degrees of freedom
print('Degrees of freedom:', n-1)

Degrees of freedom: 1469


For ⍺ = 0.05 and degrees of freedom = 1469, calculate the critical value for the left-tail.

In [23]:
# calculate the χ2-value for 90% of confidence level
# use 'stats.chi2.ppf()' to find the χ2-value corresponding to the lower tail probability 'q'
# pass the value of 'alpha' to the parameter 'q'
# pass the degrees of freedom to the parameter 'df' 
# use 'round()' to round-off the value to 4 digits
chi2_val = np.abs(round(stats.chi2.ppf(q = 0.05, df = 1469), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 1380.9949


i.e. if the chi-square value is less than 1380.9949 then we reject the null hypothesis.

In [24]:
# calculate the test statistic
chi_test = (np.sum((dist - samp_mean)**2)) / sig_2

# print the test statistic
print('Test Statistic:', chi_test)

Test Statistic: 6436.301179138322


The above output shows that the chi-square test statistic is greater than 1380.9949. Thus we reject the null hypothesis and conclude that the distance travelled by each employee to reach the office has a variance of less than 15 miles.

<a id = "1way"> </a>
## 2. One-way ANOVA

Use the data available in the CSV file `sales_emp.csv` for questions 9 to 13.

In [25]:
# read the sample data 
df_emp = pd.read_csv('sales_emp.csv')

# display the first two observations
df_emp.head(2)

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,36,Travel_Rarely,1218,9,4,Life Sciences,1,27,3,82,2,1,1,Single,3407,6986,7,Y,No,23,4,2,80,0,10,4,3,5,3,0,3
1,39,Travel_Rarely,895,5,3,Technical Degree,1,42,4,56,3,2,4,Married,2086,3335,3,Y,No,14,3,3,80,1,19,6,4,1,0,0,0


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>9. Check whether we can use the given dataset to study the equality of average monthly income of sales executives with a different education background in the company. Use a p-value technique to test at a 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In the given dataset, the `EducationField` is a categorical variable and `MonthlyIncome` is a numeric variable. To use the one-way ANOVA test for the equality of average income for all the education fields; the sample data should be drawn from normally distributed population. Also the population variances for all the groups should be equal.

Let us check the normality of the monthly income.

In [26]:
# perform Shapiro-Wilk test to test the normality
# shapiro() returns a tuple having the values of test statistics and the corresponding p-value
stat, p_value = stats.shapiro(df_emp['MonthlyIncome'])

# print the p-value
print('p-value:', p_value)

p-value: 0.05732710659503937


From the above result, we can see that the p-value is greater than 0.05, thus we can say that the monthly income is normally distributed. Thus the assumption of normality is satisfied.

Let us check the equality of variances.

In [27]:
# education fields in the data
df_emp['EducationField'].unique()

array(['Life Sciences', 'Technical Degree', 'Marketing', 'Medical',
       'Other'], dtype=object)

In [28]:
# perform Levene's test for the equality of variances 
# levene() returns a tuple having the values of test statistics and the corresponding p-value
# pass the number of defective belts from each group
stat, p_value = stats.levene(df_emp[df_emp['EducationField'] == 'Life Sciences']['MonthlyIncome'],
                             df_emp[df_emp['EducationField'] == 'Technical Degree']['MonthlyIncome'],
                             df_emp[df_emp['EducationField'] == 'Marketing']['MonthlyIncome'],
                             df_emp[df_emp['EducationField'] == 'Medical']['MonthlyIncome'],
                             df_emp[df_emp['EducationField'] == 'Other']['MonthlyIncome'])

# print the p-value 
print('P-Value:', p_value)

P-Value: 0.23476859109336565


From the above result, we can see that the p-value is greater than 0.05, thus we can say that the population variances are equal for all the groups.

Thus we can use the one-way ANOVA test to check the equality of average monthly income for all the education fields.

 <table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>10. Use the sales employees' dataset to test whether the average monthly income of sales executives with different education background is equal or not. Use a critical value method with 95% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: The average monthly income for all the education fields is the same<br>
H<sub>1</sub>: The average monthly income due to at least one education field is different

In [29]:
# number of education fields in the data
df_emp['EducationField'].nunique()

5

In [30]:
# total sample observations
len(df_emp)

54

Here t (=number of education fields) = 5, N (=total observations) = 54 

For ⍺ = 0.05 and degrees of freedom = (t-1, N-t) = (4, 49), calculate the critical value.

In [31]:
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'
# pass the value of 'alpha' to the parameter 'q', here alpha = 0.05
# pass the degrees of freedom (= 4) to the parameter 'dfn' 
# pass the degrees of freedom (= 49) to the parameter 'dfd' 
# use 'round()' to round-off the value to 4 digits
f = np.abs(round(stats.f.isf(q = 0.05, dfn = 4, dfd = 49), 4))

print('Critical value for F-test:', f)

Critical value for F-test: 2.5611


i.e. if the test statistic value is greater than 2.5611 then we reject the null hypothesis.

In [32]:
# perform one-way ANOVA
# pass the given data
test_stat, p_val = stats.f_oneway(df_emp[df_emp['EducationField'] == 'Life Sciences']['MonthlyIncome'],
                                  df_emp[df_emp['EducationField'] == 'Technical Degree']['MonthlyIncome'],
                                  df_emp[df_emp['EducationField'] == 'Marketing']['MonthlyIncome'],
                                  df_emp[df_emp['EducationField'] == 'Medical']['MonthlyIncome'],
                                  df_emp[df_emp['EducationField'] == 'Other']['MonthlyIncome'])

# print the test statistic
print('Test statistic:', test_stat)

Test statistic: 0.9546031924978221


The above output shows that the test statistic is less than 2.5611. Thus we fail to reject (i.e. accept) the null hypothesis and conclude that the average monthly income for all the education fields is the same.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>11. Can we use the given data of sales executives to check the equality of the average daily rate for different types of business travellers? Use a p-value technique to test at a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In the given dataset, the `BusinessTravel` is a categorical variable and `DailyRate` is a numeric variable. To use the one-way ANOVA test for the equality of average daily rate for all types of business travelling; the sample data should be drawn from normally distributed population. Also the population variances for all the groups should be equal.

Let us check the normality of the daily rate.

In [33]:
# perform Shapiro-Wilk test to test the normality
# shapiro() returns a tuple having the values of test statistics and the corresponding p-value
stat, p_value = stats.shapiro(df_emp['DailyRate'])

# print the p-value
print('p-value:', p_value)

p-value: 0.040250398218631744


From the above result, we can see that the p-value is greater than 0.01, thus we can say that the daily rate is normally distributed. Thus the assumption of normality is satisfied.

Let us check the equality of variances.

In [34]:
# types of business travel in the data
df_emp['BusinessTravel'].unique()

array(['Travel_Rarely', 'Non-Travel', 'Travel_Frequently'], dtype=object)

In [35]:
# perform Levene's test for the equality of variances 
# levene() returns a tuple having the values of test statistics and the corresponding p-value
# pass the number of defective belts from each group
stat, p_value = stats.levene(df_emp[df_emp['BusinessTravel'] == 'Travel_Rarely']['DailyRate'],
                             df_emp[df_emp['BusinessTravel'] == 'Travel_Frequently']['DailyRate'],
                             df_emp[df_emp['BusinessTravel'] == 'Non-Travel']['DailyRate'])

# print the p-value 
print('P-Value:', p_value)

P-Value: 0.23137904227432696


From the above result, we can see that the p-value is greater than 0.01, thus we can say that the population variances are equal for all the groups.

Thus we can use the one-way ANOVA test to check the equality of the average daily rate for all the types of business travelling.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>12. Use the parametric test to check the equality of the average daily rate for all the types of business travelling. Use a p-value technique to test the data with 99% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: The average daily rate for all the business travels is the same<br>
H<sub>1</sub>: The average daily rate for at least one type of business travel is different

In [36]:
# perform one-way ANOVA
# pass the given data
test_stat, p_val = stats.f_oneway(df_emp[df_emp['BusinessTravel'] == 'Travel_Rarely']['DailyRate'],
                                  df_emp[df_emp['BusinessTravel'] == 'Travel_Frequently']['DailyRate'],
                                  df_emp[df_emp['BusinessTravel'] == 'Non-Travel']['DailyRate'])

# print the p-value
print('p-value:', p_val)

p-value: 4.527131533131016e-06


The above output shows that the p-value is less than 0.01. Thus we reject the null hypothesis and conclude that the average daily rate for at least one type of business travel is different.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>13. Find the types of business travel for which the average daily rate is different. Use a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The previous output shows that the one-way ANOVA rejects the null hypothesis. This implies that there is at least one of the types of business travel for which the daily rate of an employee is different.

In [37]:
# number of employees for each type of business travel 
df_emp['BusinessTravel'].value_counts()

Non-Travel           18
Travel_Rarely        18
Travel_Frequently    18
Name: BusinessTravel, dtype: int64

The count of employees for each type of business travel is equal. Thus we use the Tukey hsd test for post-hoc analysis. 

In [38]:
# perform tukey's range test to compare the mean daily rate for pair of business travels
# pass the daily rate to the parameter, 'data'
# pass the type of business travel to the parameter, 'groups'
comp = mc.MultiComparison(data = df_emp['DailyRate'], groups = df_emp['BusinessTravel'])

# tukey's range test
# pass the alpha value as 0.01 to get the 99% confidence interval 
post_hoc = comp.tukeyhsd(alpha = 0.01)

# print the summary table
post_hoc.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
Non-Travel,Travel_Frequently,639.7778,0.001,289.6312,989.9243,True
Non-Travel,Travel_Rarely,242.6667,0.0972,-107.4799,592.8132,False
Travel_Frequently,Travel_Rarely,-397.1111,0.0031,-747.2577,-46.9646,True


The `reject=False` for pair (Non-Travel, Travel_Rarely) denotes that we fail to reject the null hypothesis; and conclude that the average daily rate for employees who do not travel and employees who travel rarely is same.

The `reject=True` for the pairs (Non-Travel, Travel_Frequently), (Travel_Frequently, Travel_Rarely) denotes that we reject the null hypothesis; and conclude that the average daily rate is not the same.

<a id= "non-para"></a>
## 3. Equivalent Non-parametric Test

Use the data available in the CSV file `Employee_Attrition.csv` for questions 14 to 18.

In [39]:
# read the sample data 
df_emp = pd.read_csv('Employee_Attrition.csv')

# display the first two observations
df_emp.head(2)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>14. Can we use the non-parametric test to study the equality of average monthly income for all departments in the company? Use a p-value technique to test at 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In the given dataset, the `Department` is a categorical variable and `MonthlyIncome` is a numeric variable. To use the Kruskal-Wallis H test for equality of monthly income for all the departments; the sample data should drawn from a non-normal distribution.

Let us check the normality of the monthly income.

In [40]:
# perform Shapiro-Wilk test to test the normality
# shapiro() returns a tuple having the values of test statistics and the corresponding p-value
stat, p_value = stats.shapiro(df_emp['MonthlyIncome'])

# print the p-value
print('p-value:', p_value)

p-value: 4.4031748323132235e-37


From the above result, we can see that the p-value is less than 0.01, thus we can say that the monthly income is not normally distributed. Thus the assumption of normality is not satisfied. We can use the non-parametric Kruskal-Wallis H test to check the equality of average income for all the departments.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                        <b>15. Check whether the monthly income for all the departments in a company is the same or not. Test using the critical value method at a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: The average monthly income for all the departments is the same<br>
H<sub>1</sub>: The average monthly income for at least one department is different

In [41]:
# departments in the company
df_emp['Department'].unique()

array(['Sales', 'Research & Development', 'Human Resources'], dtype=object)

In [42]:
# check the number of departments in the company
n = df_emp['Department'].nunique()

# degrees of freedom
print('Degrees of freedom:', n-1)

Degrees of freedom: 2


For ⍺ = 0.01 and degrees of freedom = 2, calculate the critical value.

In [43]:
# calculate the χ2-value for 99% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'
# pass the value of 'alpha' to the parameter 'q', here alpha = 0.01
# pass the degrees of freedom to the parameter 'df' 
# use 'round()' to round-off the value to 4 digits
chi2_val = np.abs(round(stats.chi2.isf(q = 0.01, df = 2), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 9.2103


i.e. if the chi-square value is greater than 9.2103 then we reject the null hypothesis.

In [44]:
# perform kruskal-wallis H test
# pass the monthly salary for each department
test_stat, p_val = stats.kruskal(df_emp[df_emp['Department'] == 'Sales']['MonthlyIncome'],
                                 df_emp[df_emp['Department'] == 'Research & Development']['MonthlyIncome'],
                                 df_emp[df_emp['Department'] == 'Human Resources']['MonthlyIncome'])

# print the test statistic
print('Test statistic:', test_stat)

Test statistic: 42.61858855841987


The above output shows that the test statistic is greater than 9.2103. Thus we reject the null hypothesis and conclude that the average monthly income for at least one department is different.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>16. Is it possible to identify the department with different average monthly income? If yes, perform the test at a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The Kruskal-Wallis H test has rejected the null hypothesis. Now to identify the department with a different average monthly income we can use the `conover test` to perform a post-hoc analysis.

In [45]:
# perform the conover test to identify the department with different average monthly income 
# pass the dataframe to the parameter, 'a'
# pass the column with numeric data to the parameter, 'val_col'
# pass the column with categoric data to the parameter, 'group_col'
scikit_posthocs.posthoc_conover(a = df_emp, val_col = 'MonthlyIncome', group_col = 'Department')

,Human Resources,Research & Development,Sales
Human Resources,1.000000,7.431216e-01,2.076733e-03
Research & Development,0.743122,1.000000e+00,1.084486e-10
Sales,0.002077,1.084486e-10,1.000000e+00


The p-value for pairs(Sales, Human Resources) and (Sales, Research & Development) is less than 0.01. Thus we can conclude that there is a difference in the monthly income between pairs of departments (Sales, Human Resources) and (Sales, Research & Development).

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>17. Check whether we can perform the Kruskal-Wallis H test to test the equality of the average experience of employees in all the departments. Consider a 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In the given dataset, the `Department` is a categorical variable and `TotalWorkingYears` is a numeric variable. To use the Kruskal-Wallis H test for equality of experience for all the departments; the sample data should drawn from a non-normal distribution.

Let us check the normality of the total working years of an employee.

In [46]:
# perform Shapiro-Wilk test to test the normality
# shapiro() returns a tuple having the values of test statistics and the corresponding p-value
stat, p_value = stats.shapiro(df_emp['TotalWorkingYears'])

# print the p-value
print('p-value:', p_value)

p-value: 5.62833706762415e-29


From the above result, we can see that the p-value is less than 0.05, thus we can say that the  total working years is not normally distributed. Thus the assumption of normality is not satisfied. We can use the non-parametric Kruskal-Wallis H test to check the equality of average experience for all the departments.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                        <b>18. Perform a non-parametric test to test the equality of the average experience of employees in all the departments. Use the p-value technique to perform the test with 95% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

The null and alternative hypothesis is:

H<sub>0</sub>: The average experience of employees in all the departments is the same<br>
H<sub>1</sub>: The average experience of employees in at least one department is different

In [47]:
# departments in the company
df_emp['Department'].unique()

array(['Sales', 'Research & Development', 'Human Resources'], dtype=object)

In [48]:
# perform kruskal-wallis H test
# pass the total working years for each department
test_stat, p_val = stats.kruskal(df_emp[df_emp['Department'] == 'Sales']['TotalWorkingYears'],
                                 df_emp[df_emp['Department'] == 'Research & Development']['TotalWorkingYears'],
                                 df_emp[df_emp['Department'] == 'Human Resources']['TotalWorkingYears'])

# print the p-value
print('p-value:', p_val)

p-value: 0.9313594116888158


The above output shows that the p-value is greater than 0.05. Thus we fail to reject (i.e. accpet) the null hypothesis and conclude that the average experience of employees in all the departments is the same.

<a id= "2way"></a>
## 4. Two-way ANOVA

Use the data available in the CSV file `sales_emp.csv` for questions 19 and 20.

In [49]:
# read the sample data 
df_emp = pd.read_csv('sales_emp.csv')

# display the first two observations
df_emp.head(2)

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,36,Travel_Rarely,1218,9,4,Life Sciences,1,27,3,82,2,1,1,Single,3407,6986,7,Y,No,23,4,2,80,0,10,4,3,5,3,0,3
1,39,Travel_Rarely,895,5,3,Technical Degree,1,42,4,56,3,2,4,Married,2086,3335,3,Y,No,14,3,3,80,1,19,6,4,1,0,0,0


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>19. Check whether the monthly income of employee depends on the education field and job involvement of the employee. Use the p-value technique with 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

Let us check the normality of the monthly income.

In [50]:
# perform Shapiro-Wilk test to test the normality
# shapiro() returns a tuple having the values of test statistics and the corresponding p-value
stat, p_value = stats.shapiro(df_emp['MonthlyIncome'])

# print the p-value
print('p-value:', p_value)

p-value: 0.05732710659503937


From the above result, we can see that the p-value is greater than 0.05, thus we can say that the monthly income is normally distributed. Thus the assumption of normality is satisfied.

As the job involvement level is categorical; change the data type of the variable to `object`.

In [51]:
df_emp['JobInvolvement'] = df_emp['JobInvolvement'].astype('object')

The null and alternative hypothesis for the education field is:

H<sub>0</sub>: The average monthly income for all the education fields is the same<br>
H<sub>1</sub>: At least one education field has a different average monthly income

The null and alternative hypothesis for job involvement is:

H<sub>0</sub>: The average monthly income for all the job involvement levels is the same<br>
H<sub>1</sub>: At least one job involvement level has a different average monthly income

In [52]:
# perform two-way ANOVA

# fit an ols model on the dataframe 'df_emp' 
# use 'Q()' to quote the variable name 
# use 'fit()' to fit the linear model
test = ols('MonthlyIncome ~ Q("EducationField") + Q("JobInvolvement")', df_emp).fit()

# create table for 2-way ANOVA test
# pass the linear model 'test'
# 'typ = 2' performs two-way ANOVA
anova_2 = anova_lm(test, typ = 2)

# print the table
anova_2

,sum_sq,df,F,PR(>F)
"Q(""EducationField"")",3.661317e+07,4.0,0.822651,0.517502
"Q(""JobInvolvement"")",1.274268e+07,3.0,0.381749,0.766613
Residual,5.118225e+08,46.0,NaN,NaN


The above output shows that the p-value for treatments (i.e. EducationField) is greater than 0.05. Thus we fail to reject (i.e. accept) the null hypothesis for treatments and conclude that the average monthly income for all the education fields is the same.

The p-value for blocks (i.e. JobInvolvement) is greater than 0.05. Thus we fail to reject (i.e. accept) the null hypothesis for blocks and conclude that the average monthly income for all the levels of job involvement is the same.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>20. Check whether the daily rate is affected by the types of business travel and levels of work-life balance. Use the p-value technique with 1% level of significance.</b>
                </font> 
            </div>
        </td>
    </tr>
</table>

Let us check the normality of the daily rate.

In [53]:
# perform Shapiro-Wilk test to test the normality
# shapiro() returns a tuple having the values of test statistics and the corresponding p-value
stat, p_value = stats.shapiro(df_emp['DailyRate'])

# print the p-value
print('p-value:', p_value)

p-value: 0.040250398218631744


From the above result, we can see that the p-value is greater than 0.01, thus we can say that the daily rate is normally distributed. Thus the assumption of normality is satisfied.

As the variable indicating levels of work-life balance is categorical; change the data type of the variable to `object`.

In [54]:
df_emp['WorkLifeBalance'] = df_emp['WorkLifeBalance'].astype('object')

The null and alternative hypothesis for the types of business travel is:

H<sub>0</sub>: The average daily rate for all the business travels is the same<br>
H<sub>1</sub>: At least one type of business travel has a different average daily rate

The null and alternative hypothesis for levels of work-life balance is:

H<sub>0</sub>: The average daily rate for all the work-life balance levels is the same<br>
H<sub>1</sub>: At least one work-life balance level has a different average daily rate

In [55]:
# perform two-way ANOVA

# fit an ols model on the dataframe 'df_emp' 
# use 'Q()' to quote the variable name 
# use 'fit()' to fit the linear model
test = ols('DailyRate ~ Q("BusinessTravel") + Q("WorkLifeBalance")', df_emp).fit()

# create table for 2-way ANOVA test
# pass the linear model 'test'
# 'typ = 2' performs two-way ANOVA
anova_2 = anova_lm(test, typ = 2)

# print the table
anova_2

,sum_sq,df,F,PR(>F)
"Q(""BusinessTravel"")",3.559568e+06,2.0,14.293493,0.000013
"Q(""WorkLifeBalance"")",7.806979e+04,3.0,0.208993,0.889682
Residual,5.976821e+06,48.0,NaN,NaN


The above output shows that the p-value for treatments (i.e. BusinessTravel) is less than 0.01. Thus we reject the null hypothesis for treatments and conclude that at least one type of business travel has a different average daily rate.

The p-value for blocks (i.e. WorkLifeBalance) is greater than 0.01. Thus we fail to reject (i.e. accept) the null hypothesis for blocks and conclude that the average daily rate for all the work-life balance levels is the same.